In [2]:
!pip install selenium

# 필요한 패키지 불러오기

In [3]:
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup
import datetime
from datetime import timedelta
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup as bs
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
import pickle

# 호텔별 URL을 가져오기

In [4]:
# url 함수
def make_url(Url):
    import time
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome()
    driver.get(Url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    click1 = driver.find_element(By.XPATH,'//*[@id="__next"]/div/header/section/div/div[1]/section[2]/div/div/div/div/div[2]/div/div[1]/span[1]/span')
    click1.click()
    time.sleep(6)
    click2 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/section/div/div[3]/div[4]/section/div[2]/div/span[1]')
    click2.click()
    click3 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/section/div/div[3]/div[4]/section/div[2]/div/span[2]')
    click3.click()
    time.sleep(2)
    click4 = driver.find_element(By.XPATH,'/html/body/div[12]/div/div/footer/div/button[2]')
    click4.click()
    time.sleep(2)
    #무한 스크롤
    
    time.sleep(1)
    
    SCROLL_PAUSE_TIME = 1.5
    
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
    # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
        
        last_height = new_height  

    soup = BeautifulSoup(html, 'html.parser')
    data = soup.select("section > div > div > a")
        
    for d in data:
        hotelurl.append(d.attrs['href'])
    # driver.close()
  

In [8]:
hotelurl = []
make_url("https://www.yanolja.com/hotel/r-900585?pageKey=1724955991321")

In [12]:
len(hotelurl)

44

# 리뷰 수집

In [16]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome()

hotel = []
star = []
reviews = []
rev_sum_pos = []
rev_sum_neg = []
columns = ['hotel','star' ,'reviews', 'Review Summary Pos', 'Review Summary Neg','url' ]   


for url in hotelurl:
    cnt = 0
    SCROLL_PAUSE_TIME = 0.5
    time.sleep(1)
    Url = "https://www.yanolja.com"+url
    driver.get(Url)
    WebDriverWait(driver, 10)
    # try:
    # # 요소 찾기 시도
    try : 
        rev_sum_pos.append(driver.find_element(By.CLASS_NAME, 'css-4a3h28').text)
        element = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[text()='낮은 평점 요약']")))
        element.click()
        time.sleep(1)
    except NoSuchElementException:
        rev_sum_pos.append("")
    try:   
        element = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//div[text()='낮은 평점 요약']")))
        element.click()
        time.sleep(1)
    except TimeoutException:
        pass
    try :    
        rev_sum_neg.append(driver.find_element(By.CLASS_NAME, 'css-4a3h28').text) 
    except NoSuchElementException:
        rev_sum_neg.append("") 
    # except NoSuchElementException:
    # # 요소가 없을 경우 공백 문자열 설정
    #     rev_sum_pos = ''
    #     rev_sum_neg = ''
    
    hotel.append(driver.find_element(By.CSS_SELECTOR,'#__next > div > div > main > article > div.css-1cc3d9 > div > div.css-1e56347 > div.css-11vo59c > h1').text)
    
    # 리뷰 페이지 클릭 (Click review page)
    try:
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-xa55ms'))
        )
    except NoSuchElementException :
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-ufp1xj'))
        )
    except TimeoutException:
        element2 = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.CLASS_NAME,'css-ufp1xj'))
        )
    element2.click()
    # 무한 스크롤
    import time
    time.sleep(1)
    
    # SCROLL_PAUSE_TIME = 1.5
    element = WebDriverWait(driver, 10).until(
    EC.element_to_be_clickable((By.CLASS_NAME, "css-stl2cn")))
    element.click()
    
    element = WebDriverWait(driver, 15).until(
    EC.element_to_be_clickable((By.XPATH, "//div[text()='평점 낮은순']")))
    element.click()
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")
    max = int(driver.find_element(By.CLASS_NAME,"css-12rhouj").text.replace("(","").replace(")",""))
    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
        time.sleep(SCROLL_PAUSE_TIME)
    
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if max > 1000:
            if(cnt>20):
                break
            cnt +=1
        if new_height == last_height:
            break
    
        last_height = new_height
    
    # 스크롤 끝나고 페이지 elements 가져오기
    star.append(float(driver.find_element(By.CLASS_NAME, 'css-u41ekg').text))
    soup = bs(driver.page_source, "lxml")
    
    # 특정 클래스 이름을 가진 모든 요소 찾기
    elements = soup.find_all('p')
    temp = ""
    # 데이터 추출 
    review = driver.find_elements(By.CLASS_NAME,"css-c92dc4")
    for i in review:
        temp += i.text
    reviews.append(temp)     
    
df = pd.DataFrame({"Hotel" : hotel, "Star" : star ,"Reviews" : reviews, "Review Pos" : rev_sum_pos,"Review Neg" : rev_sum_neg,"Url": hotelurl})   
df["Url"] ="https://www.yanolja.com"+df["Url"]

In [44]:
with open('gy.pickle', 'wb') as f:
    pickle.dump(df, f)

In [40]:
with open("gy.pickle", 'rb') as f:
	data = pickle.load(f)


In [42]:
data

,Hotel,Star,Reviews,Review Pos,Review Neg,Url
0,골든튤립에버용인호텔,4.7,평일에 숙박했는데도 불구하고 지하주차장이 만차라서 에버랜드 4주차장에 주차하랍니다....,"에버랜드와 가까운 위치에 있으며, 청결한 객실과 친절한 직원 서비스로 높은 만족도를...","쾌적한 시설과 편리한 위치를 갖추고 있으나, 주차 공간 부족과 직원의 불친절한 서비...",https://www.yanolja.com/hotel/3016379
1,라까사 호텔 광명,4.6,편의성모든게 실망\n다신 오고싶지않음욕실 배수가 너무 엉망이에요. 워터파크에서 샤워...,"청결하고 쾌적한 환경을 제공하며, 직원들의 친절한 서비스가 두드러집니다. 광명역과의...","객실 인테리어와 뷰가 나쁘지 않으나, 주차 서비스의 불친절한 직원과 청결 상태에서 ...",https://www.yanolja.com/hotel/3016071
2,라마다용인호텔,4.6,많은 여행을 다니면서 이런 호텔은 처음입니다\n후기가 좋아서 예약해서 방문했고 4시...,"에버랜드와 가까워 가족 단위 여행에 적합하며, 청결도가 높고 친절한 서비스가 호평받...","에버랜드와 가까운 위치에 있어 접근성이 좋으나, 주차와 방음 문제로 이용객들의 불만...",https://www.yanolja.com/hotel/3011592
3,어반스테이 동탄,4.9,일전에 한번 묵었던 숙소라 또 간건데\n건물 자체가 세입자들이랑 숙박이용객들이 섞여...,"청결 상태가 매우 우수하며, 가성비가 좋고 편안한 숙박 환경을 제공합니다. 비대면 ...","숙소는 좋은 위치와 가격으로 접근성이 뛰어나며, 편안한 숙박 환경을 제공합니다. 온...",https://www.yanolja.com/hotel/1000112790
4,테이크호텔 서울광명,4.7,진짜 너무 관리 안되는거 같아요ㄱㅅㄱㅈ없고 이상한 무칩도 망해라 폭삭 망해라ㅜ체크이...,숙소는 광명역에 인접해 있어 좋은 접근성을 제공합니다. 주변에는 다양한 음식점과 쇼...,"청소 상태와 서비스에 대한 불만이 상존하며, 고급스러운 외관과 다양한 부대시설에도 ...",https://www.yanolja.com/hotel/10041428
5,센트럴 퍼슨 평택 호텔,4.5,기본적인 세면도구를 돈주고 사야함\n아침 9시30분쯤사람이 있는데도 불구하고 문열고...,넓고 깨끗한 객실과 친절한 서비스로 고객들에게 높은 만족도를 기록하고 있습니다. 여...,"넓은 욕실과 분리된 샤워, 변기, 욕조가 특징입니다. 직원들의 친절도는 긍정적으로 ...",https://www.yanolja.com/hotel/10041257
6,리디자인 부티크호텔,4.7,전날 오전에 예약했는데 같이 가기로한 사람이 아파서 입원해서 못갔습니다. 예약한지 ...,넓고 깨끗한 객실과 친절한 직원 서비스가 돋보입니다. 주변에는 다양한 식당과 편의시...,전반적으로 청결도가 높고 친절한 직원 서비스가 특징입니다. 그러나 방음과 시설에 대...,https://www.yanolja.com/hotel/3000820
7,르컬렉티브 시흥 웨이브파크,4.7,진짜 별로입니다. 다시는 이용 안 할거예요.\n야놀자와 르컬렉티브 웨이브파크 서로가...,"청결과 가성비가 뛰어난 곳으로, 내부 시설이 잘 갖춰져 있어 편리한 숙박이 가능합니...","신축으로 청결한 시설이 장점이나, 여러 이용 후기에서 서비스와 시설의 부족함으로 인...",https://www.yanolja.com/hotel/10052429
8,코트야드 메리어트 수원,4.6,벨 몇 번 누르고 문 따고 들어와서는 안쪽에서 반응이 없어서 체크아웃 한 줄 알았다...,"숙소는 청결하고 편안한 분위기를 제공하며, 직원들의 친절함이 돋보입니다. 지하로 갤...","편의성 측면에서는 장점이 있으나, 전반적으로 쾌적함과 서비스에 대한 불만이 많습니다...",https://www.yanolja.com/hotel/10050989
9,호텔 시엘로 시흥,4.7,[호텔 사장님 절대 이렇게 운영마세요] \n1. 주말 조식pkg인데 왜 현장에서 추...,"청결하고 현대적인 시설이 갖춰져 있으며, 루프탑 수영장과 테라스에서 즐길 수 있는 ...","숙소는 가족 단위 여행객들에게 적합하며, 친절한 서비스와 깨끗한 시설이 특징입니다....",https://www.yanolja.com/hotel/10046287
